In [40]:
import pandas as pd
import numpy as np
import json
import requests
pd.set_option('display.min_rows', 20)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 100)

import datetime as datetime
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.linear_model import LinearRegression, Lasso, Ridge, LogisticRegression
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline


In [11]:
games_predict_df = pd.read_csv('team_model_2019.csv')  


games_predict_df  = games_predict_df.sort_values(by='game_number').reset_index(drop=True)

games_predict_dup_df = games_predict_df[games_predict_df.duplicated(subset=['game_number'],keep=False)]

games_predict_dup_df

,game_number,team_team_id,team_goals,team_pim,team_shots,team_powerPlayPercentage,team_powerPlayGoals,team_powerPlayOpportunities,team_faceOffWinPercentage,team_blocked,team_takeaways,team_giveaways,team_hits,team_SV_Pct,team_SH_Pct,team_CF_Pct,team_win,is_home_team,home_victory
6,2019020301,22.0,3.15,8.20,29.05,28.330,1.00,3.05,48.175,14.90,8.60,12.00,21.55,0.916649,0.109485,0.487836,0.65,1,False
7,2019020301,25.0,2.50,8.10,29.70,10.880,0.40,2.95,51.970,14.05,6.80,12.45,17.40,0.921241,0.080722,0.497560,0.50,0,False
8,2019020302,53.0,2.75,6.65,31.05,16.165,0.50,3.10,49.120,12.20,7.40,9.50,17.75,0.920472,0.091320,0.480060,0.55,1,True
9,2019020302,20.0,2.75,8.80,30.95,13.660,0.55,3.30,48.865,12.70,8.85,11.25,14.50,0.909703,0.084778,0.497508,0.45,0,True
12,2019020309,19.0,3.05,6.85,28.60,26.000,0.85,3.20,52.525,12.10,7.40,9.15,20.25,0.912632,0.102681,0.483431,0.60,1,False
13,2019020309,24.0,2.65,9.85,28.80,7.915,0.25,2.65,52.220,13.25,6.35,11.75,21.55,0.910836,0.092102,0.471234,0.45,0,False
16,2019020313,17.0,2.15,8.80,30.00,14.635,0.55,3.25,49.000,13.80,4.00,12.75,19.80,0.882822,0.068216,0.504698,0.30,0,False
17,2019020313,28.0,2.90,12.60,29.55,20.000,0.70,3.35,49.890,12.55,8.55,10.10,22.30,0.882321,0.100679,0.494136,0.40,1,False
18,2019020315,54.0,2.80,9.90,33.60,21.830,0.70,3.35,48.970,12.95,9.95,8.65,26.65,0.897971,0.083225,0.512969,0.35,1,True
19,2019020315,20.0,2.60,8.90,31.30,13.660,0.55,3.25,49.160,12.30,8.70,10.80,15.20,0.905355,0.081330,0.506149,0.40,0,True


In [19]:
# For each duplicate combine into one row

home_teams = games_predict_dup_df[games_predict_dup_df['is_home_team']==1]

home_teams

away_teams = games_predict_dup_df[games_predict_dup_df['is_home_team']==0]

away_teams

game_model = pd.concat([home_teams.reset_index(drop=True),away_teams.reset_index(drop=True)],axis=1)

game_model

,game_number,team_team_id,team_goals,team_pim,team_shots,team_powerPlayPercentage,team_powerPlayGoals,team_powerPlayOpportunities,team_faceOffWinPercentage,team_blocked,...,team_blocked,team_takeaways,team_giveaways,team_hits,team_SV_Pct,team_SH_Pct,team_CF_Pct,team_win,is_home_team,home_victory
0,2019020301,22.0,3.15,8.20,29.05,28.330,1.00,3.05,48.175,14.90,...,14.05,6.80,12.45,17.40,0.921241,0.080722,0.497560,0.50,0,False
1,2019020302,53.0,2.75,6.65,31.05,16.165,0.50,3.10,49.120,12.20,...,12.70,8.85,11.25,14.50,0.909703,0.084778,0.497508,0.45,0,True
2,2019020309,19.0,3.05,6.85,28.60,26.000,0.85,3.20,52.525,12.10,...,13.25,6.35,11.75,21.55,0.910836,0.092102,0.471234,0.45,0,False
3,2019020313,28.0,2.90,12.60,29.55,20.000,0.70,3.35,49.890,12.55,...,13.80,4.00,12.75,19.80,0.882822,0.068216,0.504698,0.30,0,False
4,2019020315,54.0,2.80,9.90,33.60,21.830,0.70,3.35,48.970,12.95,...,12.30,8.70,10.80,15.20,0.905355,0.081330,0.506149,0.40,0,True
5,2019020316,15.0,3.80,9.45,31.90,26.515,0.75,3.40,50.205,15.95,...,12.95,6.10,11.35,21.20,0.911036,0.094904,0.464255,0.45,0,True
6,2019020317,53.0,2.85,7.25,30.60,16.165,0.50,3.25,49.030,12.75,...,14.40,5.65,11.35,21.05,0.879856,0.075796,0.531288,0.40,0,True
7,2019020318,7.0,2.90,8.55,30.70,21.045,0.70,3.25,46.480,11.35,...,13.10,6.00,7.55,19.30,0.897401,0.090353,0.493361,0.35,0,False
8,2019020319,13.0,3.55,7.30,34.60,26.165,0.85,3.15,49.470,13.00,...,12.35,7.15,11.00,22.05,0.900922,0.084782,0.528235,0.55,0,True
9,2019020323,17.0,2.10,8.90,29.75,14.010,0.50,3.00,49.055,14.00,...,16.15,6.10,11.80,27.95,0.906248,0.088911,0.470885,0.45,0,False


In [51]:
col_replace_home = [w.replace('team', 'home') for w in game_model.columns[:19]]
col_replace_away = [w.replace('team', 'away') for w in game_model.columns[19:-2]]
new_col_list = col_replace_home + col_replace_away + ['is_home_team'] + ['home_victory']

game_model.columns = new_col_list

data = game_model.drop(columns=['is_home_home','is_home_team','game_number','home_home_id','away_away_id'])

data

X = data.iloc[:,:-1]
Y = data.iloc[:,-1]

X2 = X.drop(columns = ['home_victory'])

In [62]:
#Do a logigistic regression

X_train, X_val, y_train, y_val = train_test_split(X2,Y,test_size=0.2,random_state = 42)

clf2 = LogisticRegression(class_weight = 'balanced')

clf2.fit(X_train, y_train)
clf2.score(X_train,y_train)
clf2.score(X_val,y_val)

proba_df = pd.DataFrame(clf2.predict_proba(X_train),columns=['zero','one'])
c1 = pd.Series(y_train==1)
c2  = proba_df['one']>0.5

c1 = c1.reset_index(drop=True)
c2 = c2.reset_index(drop=True)

TP = proba_df[c1 & c2]
TN = proba_df[~c1 & ~c2]
FP = proba_df[~c1 & c2]
FN = proba_df[c1 & ~c2]

print('TP: ' , len(TP.index),'TN: ', len(TN.index),'FP: ',len(FP.index),'FN:',len(FN.index))


TPR = len(TP.index) / (len(TP.index) + len(FN.index))
TNR = len(TN.index) / (len(TN.index) + len(FP.index))
ACC = (len(TP.index) + len(TN.index)) / (len(TP.index) + len(TN.index) + len(FP.index) + len(FN.index))

print('TPR: ' , TPR, 'TNR: ', TNR,'ACC: ', ACC)


TP:  193 TN:  187 FP:  121 FN: 108
TPR:  0.6411960132890365 TNR:  0.6071428571428571 ACC:  0.6239737274220033


C:\Users\danny\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [63]:
from sklearn.metrics import log_loss
log_loss(y_train, clf2.predict_proba(X_train))

0.6537577465002778

In [ ]:
#Things to add in:

# More Seasons
# Check Feature importance in above
# Bagging/Boosting
# Historical MatchUps? 
# News semantics
# Stacking
# Days of rest
# xG 
# Betting Lines
# K Ratings / ELO ratings
# Strength of forwards/defence as reported by the mean Player Rating of the starting lineup
# The K Rating of home and away teams
# The Adjusted 5-on-5 Fenwick differential of home and away teams in recent games
# The Elo rating of home and away teams